# Profit分析

## 目的

予測での利益と実際の利益の差分をみてみる。

In [1]:
import os
import datetime
import numpy as np
import pandas as pd

In [2]:
current_dir = %pwd
timestamp = os.path.basename(current_dir)

In [3]:
import sys
sys.path.append('../../')
%cd '../../'

/home/tsu-nera/repo/bakuchi


In [4]:
from src.utils.trade_analysis import TradeAnalysis

%reload_ext autoreload
%autoreload 2

File doesn't exist 


In [5]:
ta = TradeAnalysis(timestamp)

In [6]:
ex1_trades = ta.get_coincheck_trades_df()
ex2_trades = ta.get_liquid_trades_df()

In [7]:
len(ex1_trades), len(ex2_trades)

(32, 32)

In [8]:
ex1_ticks = ta.get_coincheck_ticks_df()
ex2_ticks = ta.get_liquid_ticks_df()

In [9]:
ex1_ticks.head(5)

,bid,ask,tick_timestamp
timestamp,,,
2020-03-25 10:37:45,741773.0,742217.0,2020-03-25 10:37:45
2020-03-25 10:37:46,741840.0,742350.0,2020-03-25 10:37:46
2020-03-25 10:37:47,741840.0,742350.0,2020-03-25 10:37:47
2020-03-25 10:37:48,741840.0,742350.0,2020-03-25 10:37:48
2020-03-25 10:37:50,741840.0,742350.0,2020-03-25 10:37:49


In [10]:
trades = []

for ex1_row, ex2_row in zip(ex1_trades.iterrows(), ex2_trades.iterrows()):
    ex1_item = ex1_row[1]
    ex2_item = ex2_row[1]
    
    ex1_side = ex1_item["side"]
    ex1_timestamp = ex1_item["datetime"]
    ex1_amount = ex1_item["amount"]
    ex1_actual_price = ex1_item["price"]
    ex1_actual_rate = ex1_item["rate"]

    ex2_side = ex2_item["side"]
    ex2_timestamp = ex2_item["datetime"]
    ex2_amount = ex2_item["amount"]
    ex2_actual_price = ex2_item["price"]
    ex2_actual_rate = ex2_item["rate"]
    
    try:
        ex1_tick = ex1_ticks.loc[ex1_timestamp]
    except Exception:
        timestamp = ex1_timestamp + datetime.timedelta(seconds=1)
        try:
            ex1_tick = ex1_ticks.loc[timestamp]
        except Exception:
            timestamp2 = timestamp + datetime.timedelta(seconds=1)
            ex1_tick = ex1_ticks.loc[timestamp2]

    try:
        ex2_tick = ex2_ticks.loc[ex2_timestamp]
    except Exception:
        timestamp = ex2_timestamp + datetime.timedelta(seconds=1)
        try:
            ex2_tick = ex2_ticks.loc[timestamp]
        except Exception:
            timestamp2 = timestamp + datetime.timedelta(seconds=1)
            ex2_tick = ex2_ticks.loc[timestamp2]

    if ex1_side == "sell":
        ex1_tick_rate = ex1_tick.bid
    else:
        ex1_tick_rate = ex1_tick.ask
    if ex2_side == "sell":
        ex2_tick_rate = ex2_tick.bid
    else:
        ex2_tick_rate = ex2_tick.ask
        
    ex1_expected_price = ex1_tick_rate * ex1_amount
    ex2_expected_price = ex2_tick_rate * ex2_amount

    
    if ex1_side == "buy":
        ex1_expected_price = ex1_expected_price * -1
        ex1_actual_price = ex1_actual_price * -1
    if ex2_side == "buy":
        ex2_expected_price = ex2_expected_price * -1
        ex2_actual_price = ex2_actual_price * -1
        
    expected_profit = sum([ex1_expected_price, ex2_expected_price])
    actual_profit = sum([ex1_actual_price, ex2_actual_price])

    trade = {
        "timestamp": ex1_timestamp, 
        "ex1_side": ex1_side,
        "ex1_expected_rate": ex1_tick_rate,
        "ex1_actual_rate": ex1_actual_rate,
        "ex1_expected_price": ex1_expected_price,
        "ex1_actual_price": ex1_actual_price,
        "ex2_side": ex2_side,
        "ex2_expected_rate": ex2_tick_rate,
        "ex2_actual_rate": ex2_actual_rate,
        "ex2_expected_price": ex2_expected_price,
        "ex2_actual_price": ex2_actual_price,
        "expected_profit": expected_profit,
        "actual_profit": actual_profit
    }
    
    trades.append(trade)

In [11]:
trade_df = pd.DataFrame(trades)
trade_df = trade_df.set_index("timestamp")

In [12]:
profit_df = trade_df[["expected_profit", "actual_profit"]]

In [13]:
int(sum(profit_df["expected_profit"])), int(sum(profit_df["actual_profit"]))

(144, 4)

In [14]:
pd.set_option('display.max_rows', None)
profit_df

,expected_profit,actual_profit
timestamp,,
2020-03-25 10:52:19,6.984000,-1.134
2020-03-25 10:52:51,-2.069843,-1.301
2020-03-25 10:55:20,7.014000,-1.782
2020-03-25 10:58:05,4.200458,-2.561
2020-03-25 10:59:26,6.774000,-0.786
2020-03-25 11:01:36,-0.109516,-0.110
2020-03-25 11:07:55,6.186000,0.474
2020-03-25 11:08:16,5.110678,0.495
2020-03-25 11:20:25,6.234000,1.356


## coincheck buyにおける売却コインの差分分析

In [15]:
ex1_buy_trades = ex1_trades[ex1_trades["side"] == "buy"]

In [16]:
ex1_buy_amount = ex1_buy_trades["amount"]
min(ex1_buy_amount), max(ex1_buy_amount), np.mean(ex1_buy_amount)

(0.00599186, 0.00600169, 0.0059974225)

In [17]:
from decimal import Decimal
num = 0.006 - np.mean(ex1_buy_amount)
"{0:.10f}".format(Decimal(num))

'0.0000025775'

この価格差によって生じた資金移動は

In [18]:
num * len(ex1_buy_trades) * np.mean(ex1_ticks["ask"])

30.307018114283725

## rateの誤差比較

In [19]:
ex1_rate_df = trade_df[["ex1_side", "ex1_expected_rate", "ex1_actual_rate"]].copy()
ex1_rate_df["ex1_rate_diff"] = ex1_rate_df["ex1_expected_rate"] - ex1_rate_df["ex1_actual_rate"]

In [20]:
ex1_rate_df

,ex1_side,ex1_expected_rate,ex1_actual_rate,ex1_rate_diff
timestamp,,,,
2020-03-25 10:52:19,sell,743000.0,741626.0,1374.0
2020-03-25 10:52:51,buy,741919.0,741791.0,128.0
2020-03-25 10:55:20,sell,739677.0,737974.0,1703.0
2020-03-25 10:58:05,buy,733301.0,734287.0,-986.0
2020-03-25 10:59:26,sell,735582.0,734316.0,1266.0
2020-03-25 11:01:36,buy,735464.0,735464.0,0.0
2020-03-25 11:07:55,sell,736383.0,735381.0,1002.0
2020-03-25 11:08:16,buy,735247.0,736017.0,-770.0
2020-03-25 11:20:25,sell,739472.0,738681.0,791.0


In [21]:
ex1_diff = ex1_rate_df["ex1_rate_diff"]
max(ex1_diff), min(ex1_diff), np.mean(abs(ex1_diff)), np.std(ex1_diff)

(1703.0, -1202.5, 666.9375, 807.885989766463)

In [22]:
ex2_rate_df = trade_df[["ex2_side", "ex2_expected_rate", "ex2_actual_rate"]].copy()
ex2_rate_df["ex2_rate_diff"] = ex2_rate_df["ex2_expected_rate"] - ex2_rate_df["ex2_actual_rate"]

In [23]:
ex2_diff = ex2_rate_df["ex2_rate_diff"]
max(ex2_diff), min(ex2_diff), np.mean(abs(ex2_diff)), np.std(ex2_diff)

(531.0, -1387.0, 190.484375, 369.6436625398295)